# semantic similarity

First step, merge. 1.
1. The two files are named "dictionary 1-robot.xlsx" and "dictionary 2non-robot.xlsx". 
2. Please merge sheet2 of the two tables into a new table according to the following rules: 2.1.
2.1. The structure of the two tables is exactly the same, with the headers "Patent #" "publication year" "keyword from n-gram", please merge the two tables into a single table.
2.2. before merging, iterate over "patent #" if the two tables have identical "patent #" then, "publication year" keep one of them, "keyword from n-gram" merge it and space it with ";".

In [1]:
import pandas as pd

# Define the file names
file1 = "dictionary 1-robot.xlsx"
file2 = "dictionary 2non-robot.xlsx"

# Load sheet2 from both Excel files
df1 = pd.read_excel(file1, sheet_name="sheet2")
df2 = pd.read_excel(file2, sheet_name="sheet2")

# Merge the two DataFrames based on the "Patent #" column
merged_df = pd.merge(df1, df2, on="Patent #", how="outer")

# Define a function to merge the "keyword from n-gram" columns with ";"
def merge_keywords(row):
    keywords1 = str(row["keyword from n-gram_x"])
    keywords2 = str(row["keyword from n-gram_y"])
    if keywords1 == "nan":
        return keywords2
    elif keywords2 == "nan":
        return keywords1
    else:
        return keywords1 + "; " + keywords2

# Apply the function to merge the "keyword from n-gram" columns
merged_df["keyword from n-gram"] = merged_df.apply(merge_keywords, axis=1)

# Drop the redundant columns and keep one "publication year" and "PRLI"
merged_df = merged_df.drop(columns=["publication year_y", "PRLI_y", "keyword from n-gram_x", "keyword from n-gram_y"])

# Rename the columns if needed
# merged_df = merged_df.rename(columns={"publication year_x": "publication year", "PRLI_x": "PRLI"})

# Save the merged DataFrame to a new Excel file
merged_df.to_excel("merged_table.xlsx", index=False)

print("Merged table saved as 'merged_table.xlsx'")
print("Number of rows in the merged table:", len(merged_df), "\n")

Merged table saved as 'merged_table.xlsx'
Number of rows in the merged table: 779 



c:\Users\Eric\.conda\envs\lym1\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


### Step 1: Preprocess "merged_table.xlsx":

Read "merged_table.xlsx."
Process the "keyword from n-gram" column to replace underscores with spaces and split multiple keywords separated by ';' into separate rows.
Create a new dataset called "processed_keyword."
### Step 2: Preprocess "corpus_dataset.xlsx":

Read "corpus_dataset.xlsx."
Process the "Corpus" column to split words, remove stopwords, and perform common text cleaning.
Create a new dataset called "processed_corpus."
### Step 3: Calculate Semantic Similarity:

Calculate semantic similarity between "processed_keyword" and "processed_corpus" using cosine similarity, Jaccard similarity, and Euclidean distance.
Output the similarity scores.

In [3]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Ensure NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')

# Step 1: Preprocess "merged_table.xlsx"
merged_data = pd.read_excel("merged_table.xlsx")

# Create a dictionary to store processed keywords
keyword_dict = {}

for index, row in merged_data.iterrows():
    patent_id = row["Patent #"]
    keywords = row["keyword from n-gram"].split(";")
    
    # Process and clean the keywords
    cleaned_keywords = []
    for keyword in keywords:
        keyword = keyword.replace("_", " ").strip()
        keyword = keyword.replace("robots", "robot").strip()
        cleaned_keywords.append(keyword)
    
    # Store the processed keywords for each patent ID
    keyword_dict[patent_id] = " ".join(cleaned_keywords)

# Step 2: Preprocess "corpus_dataset.xlsx"
corpus_data = pd.read_excel("corpus_dataset.xlsx")

# Create a dictionary to store processed corpus text
corpus_dict = {}

for index, row in corpus_data.iterrows():
    patent_id = row["Patent #"]
    text = row["Corpus"]
    
    # Process and clean the corpus text
    if isinstance(text, str) and text.strip():
        tokens = word_tokenize(text)
        tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stopwords.words('english')]
        cleaned_text = " ".join(tokens)
        corpus_dict[patent_id] = cleaned_text

# Filter the corpus dictionary to keep only entries with matching patent IDs in keyword_dict
corpus_dict = {patent_id: text for patent_id, text in corpus_dict.items() if patent_id in keyword_dict}

# Check if there are any redundant Processed Corpus Text entries
redundant_corpus_text = [patent_id for patent_id in corpus_dict if patent_id not in keyword_dict]

# Create a table to display Patent ID, Processed Keywords, Processed Corpus Text
table_data = []
for patent_id in keyword_dict:
    if patent_id in corpus_dict:
        table_data.append([patent_id, keyword_dict[patent_id], corpus_dict[patent_id]])

# Display the table
table = pd.DataFrame(table_data, columns=["Patent ID", "Processed Keywords", "Processed Corpus Text"])

# Display the results
print("Correspondence Check Results:")
print(table)

# Ensure that the lengths of processed_keyword and processed_corpus are the same
common_patent_ids = set(keyword_dict.keys()).intersection(corpus_dict.keys())

# Check the number of common patent IDs and their uniqueness
print(f"Are the Common Patent IDs Unique? {len(common_patent_ids) == len(set(common_patent_ids))}")
print(f"Number of Common Patent IDs: {len(common_patent_ids)}")
print(f"Number of Redundant Processed Corpus Text Entries: {len(redundant_corpus_text)}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Eric\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Eric\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Correspondence Check Results:
         Patent ID                           Processed Keywords  \
0     FI198303365A                        articulated robot arm   
1      FR2547755A1               lightweight construction robot   
2       EP132176A1               lightweight construction robot   
3      JP60087983A               lightweight construction robot   
4    ES198503542A1               lightweight construction robot   
..             ...                                          ...   
774   CN218534534U                       moving track mechanism   
775   CN115599099A                      robot coordinate system   
776   CN115653251A  speed reducing motor vertical sliding plate   
777  KR2023022067A                   three dimensional printing   
778   CN115726591A                         wall building device   

                                 Processed Corpus Text  
0    articulated rotary robot arm hinge coupling li...  
1    prim advantage invention light weight thus mou


# Step 3: Calculate Semantic Similarity

WE want to compare the cosine similarity, Euclidean distance, and Jaccard similarity between the long sentences formed by combining all "Processed Keywords" and all "Processed Corpus Text" values. 

In [6]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

# Combine all "Processed Keywords" into a single long sentence
all_keywords = " ".join(table["Processed Keywords"])

# Combine all "Processed Corpus Text" into a single long sentence
all_corpus_text = " ".join(table["Processed Corpus Text"])

# Step 3: Calculate Semantic Similarity
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix_keyword = tfidf_vectorizer.fit_transform([all_keywords])
tfidf_matrix_corpus = tfidf_vectorizer.transform([all_corpus_text])

# Calculate cosine similarity
cosine_similarity_scores = cosine_similarity(tfidf_matrix_keyword, tfidf_matrix_corpus)[0][0]

# Calculate Euclidean distance
euclidean_distance_scores = euclidean_distances(tfidf_matrix_keyword, tfidf_matrix_corpus)[0][0]

# Calculate Jaccard similarity
keywords_set = set(all_keywords.split())
corpus_set = set(all_corpus_text.split())

# Calculate Jaccard similarity
jaccard_similarity = len(keywords_set.intersection(corpus_set)) / len(keywords_set.union(corpus_set))

# Output the similarity scores
print("Cosine Similarity Score:", cosine_similarity_scores)
print("Euclidean Distance Score:", euclidean_distance_scores)
print("Jaccard Similarity Score:", jaccard_similarity)


Cosine Similarity Score: 0.9147749990130831
Euclidean Distance Score: 0.412855909457324
Jaccard Similarity Score: 0.1185076810534016
